# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output = "weatherpy.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
n=700
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

610

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
# Set temperature units
temp_unit = "imperial"

# Starting URL for Weather Map API Call
url =f"https://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&units={temp_unit}&q="


# List of city data
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1
columns = ["City", "Cloudiness", "Country", "Date", "Humidity", "Lat", "Lng", "Max Temp (F)", "Wind Speed"]
city_data = pd.DataFrame(columns = columns)
# Set temperature units
temp_unit = "imperial"


# summary lists
city2 = []
cloudinesses = []
dates = []
humidities = []
lat = []
lng = []
max_temp = []
wind_speed = []
countries = []

# Set initial count
count_one = 0
set_one = 1

# Loop for creating summary
for city in cities:
   # print (city)
    #end point url for each city
    try:
        response = response = requests.get(url + city).json()
        cloudinesses.append(response['clouds']['all'])
        countries.append(response['sys']['country'])
        dates.append(response['dt'])
        humidities.append(response['main']['humidity'])
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        max_temp.append(response['main']['temp_max'])
        wind_speed.append(response['wind']['speed'])
        if count_one > 48:
            count_one = 1
            set_one += 1
            city2.append(city)
        else:
            count_one += 1
            city2.append(city)

        print(f"Processing Record {count_one} of Set {set_one} | {city}") 
    except Exception:
        print("City not found. Skipping ...")


  
print('------------------------------')
print('Retrieval Complete')
print('------------------------------')
  


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | aklavik
Processing Record 2 of Set 1 | praia
Processing Record 3 of Set 1 | ponta do sol
Processing Record 4 of Set 1 | parana
Processing Record 5 of Set 1 | rikitea


KeyboardInterrupt: 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# Create dictionary for summary dataframe
weather_dict = {"City":city2, "Country":countries, "Latitude":lat, "Longitude":lng, "Date":dates, "Cloudiness":cloudinesses, "Humidity":humidities, "Max Temperature":max_temp, "Wind Speed":wind_speed}

# Create dataframe
weather_df = pd.DataFrame(weather_dict)

# Display dataframe
weather_df

In [ ]:
#export to .csv
weather_df.to_csv(output, index_label="City")

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
# Create Temperature v. Latitude Scatter
plt.scatter(weather_df["Latitude"],weather_df["Max Temperature"],edgecolors="black",facecolors="lightblue")

# Set titles and labels
plt.title("City Latitude vs. Max Temperature (4/23/21)")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
#save image
plt.savefig("LatvTemperature.png")

# Display scatter
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
# Create Humidity v. Latitude chart
plt.scatter(weather_df["Latitude"],weather_df["Humidity"],edgecolors="black",facecolors="lightblue")

# Set labels and title
plt.title("City Latitude vs. Humidity (4/23/21)")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

# Set y limit
plt.ylim(15,105)
#save image
plt.savefig("LatvHumidity.png")

# Display scatter
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:

# Create Cloudiness v. Latitude chart
plt.scatter(weather_df["Latitude"],weather_df["Cloudiness"],edgecolors="black",facecolors="lightblue")

# Set title and labels
plt.title("City Latitude vs. Cloudiness (4/23/21)")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
#save image
plt.savefig("LatvCloudiness.png")


# Display chart
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:

# Create Wind v. Latitude chart
plt.scatter(weather_df["Latitude"],weather_df["Wind Speed"],edgecolors="black",facecolors="lightblue")

# Set labels and title
plt.title("City Latitude vs. Wind Speed (4/23/21)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")

# Set y limit
plt.ylim(-2,34)
#save image
plt.savefig("LatvWindspeed.png")


# Display chart
plt.show()

## Linear Regression

### Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x = 0
northhem = weather_df.query(f"Latitude >= {x}")
southhem = weather_df.query(f"Latitude < {x}")
# Create scatter plot and regression
x_axis = northhem["Latitude"]
y_axis = northhem["Max Temperature"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
regress = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_axis,y_axis,edgecolors="black",facecolors="lightblue")
plt.plot(x_axis,regress,"r-")

# Set labels and title
plt.annotate(line_eq,(0,270),fontsize=15,color="red")
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.title("Latitude vs. Max Temperature (Northern Hemisphere) (4/23/21)")

# Print regression
print(f"The r-value is: {rvalue}")
#save image
plt.savefig("Nhemistemplin.png")


# Display graph
plt.show()

###  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Create scatter plot and regression
x_axis = southhem["Latitude"]
y_axis = southhem["Max Temperature"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
regress = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_axis,y_axis,edgecolors="black",facecolors="lightblue")
plt.plot(x_axis,regress,"r-")

# Set labels and title
plt.annotate(line_eq,(-50,300),fontsize=15,color="red")
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.title("Latitude vs. Max Temperature (Southern Hemisphere) (4/23/21)")

# Print correlation
print(f"The r-value is: {rvalue}")
#save image
plt.savefig("SHemitemvlatlinear.png")


# Display graph
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:

# Create scatter plot and regression
x_axis = northhem["Latitude"]
y_axis = northhem["Humidity"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
regress = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_axis,y_axis,edgecolors="black",facecolors="lightblue")
plt.plot(x_axis,regress,"r-")

# Set labels and title
plt.annotate(line_eq,(60,100),fontsize=15,color="red")
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.title("Latitude vs. Humidity (Northern Hemisphere) (4/23/21)")

# Print regression
print(f"The r-value is: {rvalue}")

#save image
plt.savefig("NHemihumidlinear.png")

# Display graph
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:

# Create scatter plot and regression
x_axis = southhem["Latitude"]
y_axis = southhem["Humidity"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
regress = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_axis,y_axis,edgecolors="black",facecolors="lightblue")
plt.plot(x_axis,regress,"r-")

# Set labels and title
plt.annotate(line_eq,(-50,30),fontsize=15,color="red")
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.title("Latitude vs. Humidity (Southern Hemisphere) (4/23/21)")

# Print regression
print(f"The r-value is: {rvalue}")
#save image
plt.savefig("ShemiHumidlinear.png")


# Display graph
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:

# Create scatter plot and regression
x_axis = northhem["Latitude"]
y_axis = northhem["Cloudiness"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
regress = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_axis,y_axis,edgecolors="black",facecolors="lightblue")
plt.plot(x_axis,regress,"r-")

# Set labels and title
plt.annotate(line_eq,(0,-10),fontsize=15,color="red")
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.title("Latitude vs Cloudiness (Northern Hemisphere)(4/23/21)")

# Print regression
print(f"The r-value is: {rvalue}")

#save image
plt.savefig("Nhemicloudlinear.png")

# Display graph
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:

# Create scatter plot and regression
x_axis = southhem["Latitude"]
y_axis = southhem["Cloudiness"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
regress = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_axis,y_axis,edgecolors="black",facecolors="lightblue")
plt.plot(x_axis,regress,"r-")

# Create labels and title
plt.annotate(line_eq,(-50,-10),fontsize=15,color="red")
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.title("Latitude vs. Cloudiness (Southern Hemisphere)(4/23/21)")

# Print regression
print(f"The r-value is: {rvalue}")

#save image
plt.savefig("SHemicloudlinear.png")

# Display graph
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:

# Create scatter plot and regression
x_axis = northhem["Latitude"]
y_axis = northhem["Wind Speed"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
regress = x_axis * slope + intercept
plt.scatter(x_axis,y_axis,edgecolors="black",facecolors="lightblue")
plt.plot(x_axis,regress,"r-")

# Set labels and title
plt.annotate(line_eq,(-1,-1),fontsize=15,color="red")
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.title("Latitude vs. Wind Speed (Northern Hemisphere) (4/23/21)")

# Print regression
print(f"The r-value is: {rvalue}")
#save image
plt.savefig("NHemiwindlinear.png")

# Display graph
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:

# Create scatter plot and regression
x_axis = southhem["Latitude"]
y_axis = southhem["Wind Speed"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
regress = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_axis,y_axis,edgecolors="black",facecolors="lightblue")
plt.plot(x_axis,regress,"r-")

# Set labels and title
plt.annotate(line_eq,(-50,-1),fontsize=15,color="red")
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.title("Latitude vs. Wind Speed (Southern Hemisphere)(4/23/21)")

# Print regression
print(f"The r-value is: {rvalue}")
#save image
plt.savefig("Shemicloudlinear.png")


# Display graph
plt.show()